In [ ]:
import warnings, os
import datetime as dt
import itertools as it
from numpy import nan
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

# Load

In [ ]:
Raw = pd.read_csv('Storage/Metatrader_WINN.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin([*SYMBOLS]) & Raw['tf'].isin([*TFRAMES])] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$N'], TFRAMES=['M5','M15','H1','H3','D1','W1'])
Query.head()

# Data

In [ ]:
def FORMAT_SOURCE(Raw):
    Src = pd.DataFrame()
    Src['A']        = Raw['a']
    Src['Z']        = Raw['z']
    Src['Symbol']   = Raw['symbol']
    Src['TF']       = Raw['tf']
    
    Src['Datetime'] = Raw['datetime']
    Src['Date']     = Raw['datetime'].dt.date
    Src['Time']     = Raw['datetime'].dt.time
    
    Src['Ticks']    = Raw['tick_volume']
    Src['Volume']   = Raw['real_volume']
    Src['Price']    = Raw['close']
    
    Src['Open']     = Raw['open']
    Src['High']     = Raw['high']
    Src['Low']      = Raw['low']
    Src['Close']    = Raw['close']

    Src['Change']       = (Src['Close'] - Src['Open'])
    Src['Hilo']         = (Src['High']  - Src['Low'])
    
    Src['Sign Bull']    = Src['Change'].apply(lambda x: +1 if (x >= 0) else nan)
    Src['Sign Bear']    = Src['Change'].apply(lambda x: -1 if (x <  0) else nan)
    return Src 

Src = FORMAT_SOURCE(Raw=Query)
Src.head()

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc['Chg Abs'] = Calc['Change'].abs()
    Calc['Chg Pos'] = Calc['Chg Abs'] * Calc['Sign Bull']
    Calc['Chg Neg'] = Calc['Chg Abs'] * Calc['Sign Bear']
    
    Calc['HL Pos']  = Calc['Hilo'] * Calc['Sign Bull']
    Calc['HL Neg']  = Calc['Hilo'] * Calc['Sign Bear']
    return Calc

Calc = CALCULATIONS(Src)
Calc.head()

# Stats

In [15]:
Calc.groupby(['Symbol','TF'], sort=0)[['Change']].describe().round(0).astype(int)

Change                                       
            count mean   std   min   25% 50%   75%   max
Symbol TF                                               
WIN$N  M5   99999    0   116 -2055   -55   0    55  2705
       M15  46107   -1   208 -2280  -105   0   105  3655
       H1   11867   -2   411 -2875  -210   0   210  4280
       H3    4412   -6   677 -3685  -325   5   325  4620
       D1    1245  -21  1313 -5150  -850  50   830  4535
       W1     261  131  2723 -8060 -1615 -50  2045  7670

In [16]:
Calc.groupby(['Symbol','TF'], sort=0)[['Chg Abs']].describe().round(0).astype(int)

Chg Abs                                       
             count  mean   std min  25%   50%   75%   max
Symbol TF                                                
WIN$N  M5    99999    80    83   0   25    55   105  2705
       M15   46107   146   148   0   45   105   195  3655
       H1    11867   293   288   0   95   210   395  4280
       H3     4412   480   478   0  134   325   680  4620
       D1     1245  1030   813   0  420   845  1440  5150
       W1      261  2141  1683   0  855  1850  3025  8060

In [17]:
Calc.groupby(['Symbol','TF'], sort=0)[['Chg Pos']].describe().round(0).astype(int)

Chg Pos                                        
             count  mean   std min   25%   50%   75%   max
Symbol TF                                                 
WIN$N  M5    51552    78    84   0    25    55   105  2705
       M15   23412   143   150   0    45   100   195  3655
       H1     6011   287   288   0    90   210   390  4280
       H3     2255   464   466   0   130   315   670  4620
       D1      642   978   786   0   385   800  1392  4535
       W1      128  2316  1724   0  1035  2072  3178  7670

In [18]:
Calc.groupby(['Symbol','TF'], sort=0)[['Chg Neg']].describe().round(0).astype(int)

Chg Neg                                       
             count  mean   std   min   25%   50%  75% max
Symbol TF                                                
WIN$N  M5    48447   -83    82 -2055  -110   -60  -30  -5
       M15   22695  -148   146 -2280  -200  -105  -50  -5
       H1     5856  -299   288 -2875  -405  -210  -95  -5
       H3     2157  -497   489 -3685  -700  -340 -135  -5
       D1      603 -1086   838 -5150 -1478  -885 -472  -5
       W1      133 -1972  1631 -8060 -2740 -1610 -735 -30

In [19]:
Calc.groupby(['Symbol','TF'], sort=0)[['Hilo']].describe().round(0).astype(int)

Hilo                                           
            count  mean   std   min   25%   50%   75%    max
Symbol TF                                                   
WIN$N  M5   99999   166   104     0   100   140   205   3500
       M15  46107   303   183     0   180   260   375   6160
       H1   11867   599   349    75   360   515   755   6160
       H3    4412   959   599    75   535   850  1270   6355
       D1    1245  1978   811   535  1425  1795  2370   7400
       W1     261  4480  1622  1580  3280  4115  5360  12400

In [20]:
Calc.groupby(['Symbol','TF'], sort=0)[['HL Pos']].describe().round(0).astype(int)

HL Pos                                          
            count  mean   std   min   25%   50%   75%   max
Symbol TF                                                  
WIN$N  M5   51552   164   106     0    95   140   200  3500
       M15  23412   300   188     0   180   255   370  6160
       H1    6011   590   354    75   355   505   745  6160
       H3    2255   934   594    75   515   835  1225  6355
       D1     642  1913   788   535  1395  1735  2289  7400
       W1     128  4532  1646  1580  3284  4285  5545  9635

In [21]:
Calc.groupby(['Symbol','TF'], sort=0)[['HL Neg']].describe().round(0).astype(int)

HL Neg                                           
            count  mean   std    min   25%   50%   75%   max
Symbol TF                                                   
WIN$N  M5   48447  -169   102  -2095  -210  -145  -100   -10
       M15  22695  -307   179  -2675  -380  -265  -185   -40
       H1    5856  -607   345  -3380  -765  -530  -365   -90
       H3    2157  -985   604  -4390 -1310  -865  -555   -95
       D1     603 -2047   828  -5720 -2442 -1835 -1462  -610
       W1     133 -4430  1603 -12400 -5300 -4065 -3265 -1980

In [22]:
raise Exception('STOP')

Exception: STOP

# Chart

In [ ]:
for SYMBOL, Asset in Src.groupby('Symbol', sort=0):

    plt.title(f'{SYMBOL} - Candle Size Distribuition', fontweight='bold')

    for TF, Data in Asset.groupby('TF', sort=0): 
        # sns.kdeplot(Data['Change'], label=TF)
        # sns.histplot(Data['Change'], label=TF, stat='count', bins=100, element='step', fill=False)
        plt.hist(Data['Change'], bins=100, label=TF, histtype='step')


    plt.ylabel('Frequency')
    plt.xlabel('Candle Size')

    plt.yscale('log', base=10)
    plt.gca().yaxis.set_major_formatter(mpl.ticker.ScalarFormatter())

    plt.legend()
    plt.tight_layout()
    plt.show()

# Snippets

In [ ]:
def FORMAT_SOURCE(Raw):
    Src = pd.DataFrame()
    Src['Direct']       = (Src['Close'] - Src['Open']).apply(lambda x: -1 if (x <  0) else  +1)
    Src['Bull Sign']    =                Src['Direct'].apply(lambda x:  x if (x >= 0) else nan)
    Src['Bear Sign']    =                Src['Direct'].apply(lambda x:  x if (x <  0) else nan)
    
    Src['Change']       =  Src['Close'] - Src['Open']
    Src['HL']           =  Src['High']  - Src['Low']
    return Src 


def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc['Change Abs']  = Src['Change'].abs()
    Calc['Change Pos']  = Src['Change'].apply(lambda x: x if (x >= 0) else nan)
    Calc['Change Neg']  = Src['Change'].apply(lambda x: x if (x <  0) else nan)
    Calc['HL Pos']      = Src['Bull Sign'] * Src['HL']
    Calc['HL Neg']      = Src['Bear Sign'] * Src['HL']
    return Calc
  
  
Calc.groupby(['Symbol','TF'], sort=0)[['HL']].describe().round(0).astype(int)
Calc.groupby(['Symbol','TF'], sort=0)[['HL Pos']].describe().round(0).astype(int)
Calc.groupby(['Symbol','TF'], sort=0)[['HL Neg']].describe().round(0).astype(int)
Calc.groupby(['Symbol','TF'], sort=0)[['Change']].describe().round(0).astype(int)
Calc.groupby(['Symbol','TF'], sort=0)[['Change Abs']].describe().round(0).astype(int)
Calc.groupby(['Symbol','TF'], sort=0)[['Change Pos']].describe().round(0).astype(int)
Calc.groupby(['Symbol','TF'], sort=0)[['Change Neg']].describe().round(0).astype(int)